In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from numpy.testing import assert_array_almost_equal
import statsmodels.api as sm
from line_profiler import LineProfiler

## Generating some data

In [2]:
def generate_random_data(n_rows, n_cols, seed=1):
    """ Generate random gaussian data with a given seed """
    np.random.seed(seed)
    random_data = np.random.normal(size=n_cols * n_rows, scale=4)
    random_data = random_data.reshape(n_rows, n_cols)
    random_df = pd.DataFrame(random_data)
    return random_df

In [3]:
df = generate_random_data(n_rows=64, n_cols=50)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,6.497381,-2.447026,-2.112687,-4.291874,3.461631,-9.206155,6.979247,-3.044828,1.276156,-0.997482,...,-0.767342,-3.550516,-2.988633,6.769818,0.203231,-2.547983,0.763662,8.401021,0.480636,2.468812
1,1.200681,-1.408999,-4.570073,-1.397371,-0.835577,2.346493,3.355934,3.724408,1.142349,3.540565,...,4.795672,0.740626,-1.501140,-2.554922,1.693977,0.309360,-1.375415,0.174387,-2.480003,2.792128
2,-1.788514,4.898031,1.613967,2.374314,-4.379647,0.677530,2.962226,-3.814802,-1.064874,0.130458,...,-0.746279,-0.406983,3.475545,3.001647,2.117861,0.550805,0.311285,2.473521,0.929978,2.730206
3,-1.240467,-9.739351,4.155298,8.747919,1.765458,-0.400621,-0.545779,-0.476217,0.069638,-4.488075,...,4.641354,1.477971,7.618635,4.444227,2.636199,-6.509753,2.409277,1.681129,3.243807,4.177768
4,-1.603513,3.296022,-2.249222,7.819512,-5.327807,-7.042754,-6.602885,-3.562222,-4.476462,7.824316,...,-1.992854,-1.243940,-0.007566,-5.586482,-3.445265,2.698846,2.474157,-1.772688,7.242140,-5.222908
5,-1.379949,-0.923359,-11.172340,7.750115,1.465328,-4.178358,8.204694,2.342648,1.718105,-2.427994,...,-0.746316,-0.227298,1.969346,-2.722713,-0.338032,-1.189448,1.669208,3.139083,-3.821701,2.343642
6,8.263133,-5.884628,-3.320688,-3.522310,-1.116391,6.491396,0.053411,-2.778774,2.487214,-2.399218,...,-1.646557,5.114112,-1.768917,1.294109,-0.439966,0.034196,-0.672795,-0.696721,1.844656,-4.703931
7,4.040509,3.680072,-0.780229,3.221574,-2.805378,-2.148892,0.625055,-0.760884,-1.794952,-2.689792,...,-5.706222,7.071840,-1.901492,1.910441,-4.087544,3.178113,-7.492644,3.682460,-0.141472,8.442420
8,-5.226136,0.305522,1.468927,4.931597,-1.691428,0.345858,-8.569867,-3.320675,1.806464,4.416697,...,1.805136,-6.736240,-4.640680,5.400427,-1.325133,1.546157,-3.405823,4.003526,-1.539329,5.832433
9,-2.128936,4.472534,2.697584,-2.889568,4.395985,-3.606538,-3.289869,2.886845,-2.501368,-2.375372,...,-0.421379,2.520783,-1.659388,1.807784,-6.316625,-3.314512,2.115519,-8.948346,-4.430850,-0.070873


### Ordinary Least Squares in 3 different ways

In [4]:
def ols_lstsq(row):
    """ Obtain Slope using Numpy lstsq function """
    lenght_x = row.shape[0]
    X = np.arange(lenght_x)
    ones = np.ones(lenght_x)
    X = np.vstack((X, ones)).T
    slope, intercept = np.linalg.lstsq(X, row.values, rcond=-1)[0]
    return slope


def ols_sklearn(row):
    """ Obtain Slope using sklearn LinearRegression """
    model = LinearRegression()
    lenght_X = row.shape[0]
    X = np.arange(lenght_X).reshape(-1, 1)
    model.fit(X, row.values)
    slope = model.coef_[0]
    return slope


def ols_sm(row):
    """ Obtain Slope using statsmodels.OLS """
    lenght_x = row.shape[0]
    X = np.arange(lenght_x)
    X = sm.add_constant(X)
    Y = row.values
    model = sm.OLS(Y, X)
    model_results = model.fit()
    intercept, slope = model_results.params.squeeze()
    return slope

In [5]:
%%timeit
df.apply(ols_lstsq, axis=1)

4.55 ms ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
df.apply(ols_sklearn, axis=1)

18.5 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
df.apply(ols_sm, axis=1)

22.4 ms ± 501 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
results_lstsq = df.apply(ols_lstsq, axis=1)
results_sklearn = df.apply(ols_sklearn, axis=1)
results_sm = df.apply(ols_sm, axis=1)
assert_array_almost_equal(results_sklearn, results_lstsq)
assert_array_almost_equal(results_sklearn, results_sm)

In [9]:
%%timeit
slopes = []
for row_idx in range(df.shape[0]):
    row = df.iloc[row_idx]
    slope = ols_lstsq(row)
    slopes.append(slope)
slopes = pd.Series(slopes)

10.5 ms ± 100 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
# lp = LineProfiler()
# model = LinearRegression()
# lp.run("model.fit(X, row.values)")

### Using parameter "raw"

In [11]:
def ols_lstsq_raw(row):
    """ Obtain Slope using sklearn LinearRegression, assuming Numpy array"""
    lenght_x = row.shape[0]
    X = np.arange(lenght_x)
    ones = np.ones(lenght_x)
    X = np.vstack((X, ones)).T
    slope, intercept = np.linalg.lstsq(X, row, rcond=-1)[0]
    return slope

In [12]:
%%timeit
df.apply(ols_lstsq_raw, axis=1, raw=True)

3.57 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
results_lstsq_raw = df.apply(ols_lstsq_raw, axis=1, raw=True)
assert_array_almost_equal(results_lstsq_raw, results_lstsq)

### Numba

"Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code." (https://numba.pydata.org/)

JIT: Just In Time compilation

In [14]:
import numba

@numba.jit(nopython=True)
def ols_lstsq_raw_numba(row):
    """ """
    lenght_x = row.shape[0]
    X = np.arange(lenght_x)
    ones = np.ones(lenght_x)
    X = np.vstack((X, ones)).T
    slope, intercept = np.linalg.lstsq(X, row, rcond=-1)[0]
    return slope

# this is for precompiling
_ = df.iloc[:1].apply(ols_lstsq_raw_numba, axis=1, raw=True)

In [15]:
%%timeit
df.apply(ols_lstsq_raw_numba, axis=1, raw=True)

690 µs ± 6.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
results_lstsq_raw_numba = df.apply(ols_lstsq_raw, axis=1, raw=True)
assert_array_almost_equal(results_lstsq_raw, results_lstsq_raw_numba)

### Dask

- Pandas and Numpy distributed computing
- Bag (standard Python collections), Array(NumPy) and Distributed DataFrame (Pandas)
- Super-easy parallelised Pandas functions

Dask official documentation: https://docs.dask.org/en/latest/dataframe.html

In [16]:
import dask.dataframe as dd

In [17]:
%%timeit
N_PARTITIONS = 40
SCHEDULER = "processes"
ddf = dd.from_pandas(df, npartitions=N_PARTITIONS, sort=False)
slopes_apply_raw_numba_and_dask = ddf.apply(
    ols_lstsq_raw_numba,
    axis=1,
    meta=(None, 'float64',),
    raw=True,
).compute(scheduler=SCHEDULER)

124 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Using groupby

In [42]:
# long_df = generate_random_data(n_rows=64*50, n_cols=1)
long_df = pd.DataFrame(df.values.reshape(64*50, 1))
long_df['group'] = np.arange(len(long_df)) % 64

In [43]:
%%timeit
long_df.groupby('group')[0].apply(ols_lstsq)

11.3 ms ± 279 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
long_df.groupby('group')[0].agg(ols_lstsq)

5.59 ms ± 44.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
slopes_groupby = long_df.groupby('group')[0].agg(ols_lstsq)
slopes_groupby.shape

(64,)

In [35]:
%%timeit
long_df.groupby('group')[0].transform(ols_lstsq)

21.3 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
groupby_apply_ols_lstsq = long_df.groupby('group')[0].apply(ols_lstsq)
groupby_agg_ols_lstsq = long_df.groupby('group')[0].agg(ols_lstsq)
assert_array_almost_equal(results_lstsq_raw, groupby_apply_ols_lstsq)

AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 64 / 64 (100%)
Max absolute difference: 0.14416258
Max relative difference: 90.26660527
 x: array([ 0.033202, -0.008917,  0.032409,  0.056797, -0.000817, -0.003794,
       -0.064011,  0.003285,  0.051034, -0.031013, -0.015781,  0.023516,
        0.002672, -0.010339, -0.018563, -0.018817,  0.012041, -0.085755,...
 y: array([-0.03743 ,  0.054157,  0.004054,  0.057482,  0.004816,  0.015987,
       -0.02275 , -0.03391 , -0.006535,  0.000858, -0.040914,  0.006836,
       -0.036445, -0.010995, -0.062595,  0.00456 ,  0.064367, -0.05858 ,...

In [ ]:
def get_group_ixs(group_ids):
    result = dict()
    for key in set(group_ids):
        result[key] = []
    for i, val in enumerate(group_ids):
        result[val].append(i)
    return result


def group_apply(values, group_ids, func):
    output = np.repeat(np.nan, len(values))
    ixs = get_group_ixs(group_ids)
    for ix in ixs.values():
        output[ix] = func(values[list(ix)])
    return output

In [ ]:
%%timeit
group_apply(long_df[0].values[0:1], long_df['group'].values[0:1], ols_lstsq_raw)

In [ ]:
%%timeit
group_apply(long_df[0].values, long_df['group'].values, ols_lstsq_raw_numba)

## Trying with more data

### Web Traffic Time Series Forecasting

Forecast future traffic to Wikipedia pages

Data: https://www.kaggle.com/c/web-traffic-time-series-forecasting/data

In [ ]:
# real data
df = pd.read_csv("wikipedia_train.csv")
df = df.drop("Page", axis=1)
df = df.fillna(df.median().median())
df.shape

In [ ]:
%%timeit
results_lstsq = df.apply(ols_lstsq_raw_numba, axis=1, raw=True)

In [ ]:
%%timeit
N_PARTITIONS = 32
SCHEDULER = "processes"
ddf = dd.from_pandas(df, npartitions=N_PARTITIONS, sort=False)
slopes_apply_raw_numba_and_dask = ddf.apply(
    ols_lstsq_raw_numba,
    axis=1,
    meta=(None, 'float64',),
    raw=True,
).compute(scheduler=SCHEDULER)

In [ ]:
df.shape

In [ ]:
import pandas as pd
df_long2 = pd.DataFrame(df.values.reshape(145063*550, 1))